In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc

In [2]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/avisos_uri.csv")

In [3]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/vistas_uri.csv")

In [4]:
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/postulaciones_uri.csv")

In [5]:
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/postulantes_uri.csv")
postulantes.head()

,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario
0,NM5M,FEM,47.0,-,-,-,-,Graduado,-,-
1,5awk,FEM,55.0,-,-,-,-,Graduado,Graduado,Graduado
2,ZaO5,FEM,39.0,-,-,-,-,-,Graduado,Abandonado
3,NdJl,MASC,49.0,-,-,-,En Curso,Graduado,-,Graduado
4,eo2p,MASC,37.0,-,Graduado,-,-,Graduado,-,Graduado


In [6]:
len(postulantes)

504407

In [7]:
postulantes.drop(['edad','sexo','Doctorado', 'Master','Otro', 'Posgrado','Secundario','Terciario/Técnico','Universitario'], axis=1, inplace=True)
postulantes.head()

,idpostulante
0,NM5M
1,5awk
2,ZaO5
3,NdJl
4,eo2p


In [8]:
print(len(postulantes))

504407


In [9]:
postulaciones['sepostulo'] = 1

In [10]:
no_postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/np_uri.csv")

In [11]:
postulaciones=postulaciones.append(no_postulaciones, ignore_index=True)
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))
postulaciones.head()

13200514


,Unnamed: 0,dia_postulacion,hora_postulaciones,idaviso,idpostulante,mes_postulacion,sepostulo
0,0.0,15,16,1112257047,NM5M,1,1
1,1.0,6,9,1111920714,NM5M,2,1
2,2.0,22,9,1112346945,NM5M,2,1
3,3.0,22,9,1112345547,NM5M,2,1
4,4.0,25,18,1112237522,5awk,1,1


In [12]:
postulaciones.drop('Unnamed: 0', axis=1, inplace=True)

In [13]:
postulaciones.head()

,dia_postulacion,hora_postulaciones,idaviso,idpostulante,mes_postulacion,sepostulo
0,15,16,1112257047,NM5M,1,1
1,6,9,1111920714,NM5M,2,1
2,22,9,1112346945,NM5M,2,1
3,22,9,1112345547,NM5M,2,1
4,25,18,1112237522,5awk,1,1


In [14]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')

In [15]:
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')

In [16]:
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,dia_postulacion,hora_postulaciones,idaviso,idpostulante,mes_postulacion,sepostulo,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,Unnamed: 0,mes_vista,dia_vista,hora_vistas
4847932,6,13,1112215653,12QArL,2,0,1,Técnicos Radiólogos- Eventuales,"<p><span lang=""ES""><strong>Importante Sanatori...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Salud,Unión Personal,NaN,NaN,NaN,NaN
2380673,21,19,1111937160,YWxrGV,4,0,1,Promotor de Trade Marketing - Corrientes,<p>Cervecería y Maltería Quilmes es una de las...,Gran Buenos Aires,NaN,NaN,Full-time,Junior,Ventas,AB InBev - Cervecería y Maltería Quilmes,NaN,NaN,NaN,NaN
3410185,9,17,1112430277,DrpwLra,4,1,1,Venderores / Ejecutivos Comerciales,"<p> </p><p style=""""><span style="""">Provincia S...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ventas,Provincia Seguros,4001487.0,4.0,9.0,21.0
8707834,26,22,1112425147,pzeDBLW,3,1,1,OPERADOR/A DE MONITOREO,<p>Con experiencia comprobable en monitoreo sa...,Gran Buenos Aires,NaN,NaN,Full-time,Otro,Servicios,AM SEGURIDAD EMPRESARIA S.R.L,6333459.0,3.0,27.0,2.0
1369204,27,8,1112350030,E83mz4,2,1,1,Asistente de Publicidad,"<p>Trilenium Casino, es el centro de entreteni...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Marketing,Casino Trilenium,884166.0,2.0,28.0,20.0
10413046,2,24,1112393995,pzjd25Z,4,0,1,Analista funcional Senior,"<p> </p><p style="""">En <strong style="""">TeamSe...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Tecnologia / Sistemas,TeamSeele | IT Staffing & Recruiting,NaN,NaN,NaN,NaN
8482839,15,11,1112466773,LNKkG5Q,1,0,1,Telemarketer con Experiencia - Disponibilidad ...,<p>¿Buscas un trabajo de 6 horas? ¿Queres trab...,Gran Buenos Aires,NaN,NaN,Part-time,Otro,Call Center,APPLICA - Humanos con Recursos,NaN,NaN,NaN,NaN
12651282,5,15,1112393598,QNP8Adz,3,0,1,PL- Inspector de calidad - Industria Flexográf...,<p>Importante empresa de impresión flexográfic...,Gran Buenos Aires,NaN,NaN,Full-time,Junior,Calidad,Grupo Gestión,NaN,NaN,NaN,NaN
5524967,27,10,1112362663,aWVNvm,2,1,1,Marketing Data Analyst,"<p style=""""><span style="""">Compañía líder en e...",Gran Buenos Aires,NaN,NaN,Full-time,Otro,Marketing,RH Talentum,NaN,NaN,NaN,NaN
4827248,4,10,1112379475,6zAMP4,2,0,1,Analista Senior de Compliance,<div>En <strong><em>Cohen Aliados Financieros<...,Capital Federal,NaN,Microcentro,Full-time,Senior / Semi-Senior,Auditoría,Cohen Soc. de Bolsa,NaN,NaN,NaN,NaN


In [17]:
#postulaciones.drop(['idpais', 'titulo', 'descripcion','nombre_zona','ciudad', 'mapacalle', 'tipo_de_trabajo', 'nivel_laboral', 'nombre_area', 'denominacion_empresa','Unnamed: 0'], axis=1, inplace=True)
postulaciones.sample(n=20)

,dia_postulacion,hora_postulaciones,idaviso,idpostulante,mes_postulacion,sepostulo,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,Unnamed: 0,mes_vista,dia_vista,hora_vistas
9671934,21,9,1111459399,xkdXYxY,2,1,1,Operario de Producción,<p><strong>Adecco Industrial </strong>está esp...,Gran Buenos Aires,NaN,NaN,Full-time,Junior,Producción,Adecco - Región NORTE & OESTE GBA,NaN,NaN,NaN,NaN
12603684,16,16,1112408779,ow2V8qZ,2,0,1,Encargado de Proveedores,"<p style=""""> </p><p style=""""><span style="""">Su...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Administración,Suma Recursos Humanos,NaN,NaN,NaN,NaN
10265210,5,4,1112338122,VNJe44z,2,0,1,Repositor externo,"<p style="""">Para importate industria, seleccio...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Abastecimiento,RANDSTAD,NaN,NaN,NaN,NaN
4764675,18,16,1112407310,YwGY3w,3,1,1,Administrativo,"<p align=""center"" style=""""><span style="""">Adec...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Administración,Adecco -Región Office,NaN,NaN,NaN,NaN
11492375,14,9,1112429974,4rPB3DE,4,1,1,Supervisor Operativo - Campana/Zárate,<p>GRUPO ABANS se encuentra en la búsqueda de ...,Gran Buenos Aires,NaN,NaN,Full-time,Jefe / Supervisor / Responsable,Logística,Abans Empresa de Servicios Eventuales SRL,1294532.0,4.0,15.0,10.0
3332952,10,16,1112390779,LNKMKJQ,3,1,1,Recepcionista Eventual - CABA,"<p>Para importante empresa, nos encontramos en...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Recepcionista,Manpower,NaN,NaN,NaN,NaN
9372497,27,12,1112274039,mzdaJG9,3,0,1,Desarrollador Senior JAVA (La Plata),<p>Somos una compañía multinacional líder en s...,Capital Federal,NaN,"Diagonal 80, 942",Full-time,Senior / Semi-Senior,Programación,Accenture,NaN,NaN,NaN,NaN
10197322,13,11,1112423825,vVe6Mr1,4,1,1,Analista - Project Manager Ssr,<p>Nos encontramos en la búsqueda de un<strong...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Análisis Funcional,NovaSearch,1881606.0,4.0,13.0,15.0
1607284,10,13,1112448000,bOV5jA2,4,1,1,j Cajeros - ZONA CABA,<p>Importante cadena de hipermercados incorpor...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ventas,Grupo Gestión,NaN,NaN,NaN,NaN
13900760,19,9,1112338061,xkdqlNz,2,1,1,REPOSITOR EVENTUAL CABA Y ZONA NORTE,<p><strong>IMPORTANTE SUPERMECADO DE TRAYECTOR...,Gran Buenos Aires,NaN,NaN,Part-time,Otro,Almacén / Depósito / Expedición,Asoko Tempo SA,NaN,NaN,NaN,NaN


In [18]:
postulaciones['distancia_dia'] = postulaciones['dia_postulacion'] - postulaciones['dia_vista']

In [19]:
postulaciones['distancia_hora'] = postulaciones['hora_postulaciones'] - postulaciones['hora_vistas']

In [20]:
postulaciones['distancia_mes'] = (postulaciones['mes_postulacion'] - postulaciones['mes_vista'])*30

In [21]:
postulaciones['distancia_dia'] = postulaciones['distancia_dia'].fillna(100)
postulaciones['distancia_mes'] = postulaciones['distancia_mes'].fillna(100)
postulaciones['distancia_hora'] = postulaciones['distancia_hora'].fillna(100)
postulaciones.sample(n=20)

,dia_postulacion,hora_postulaciones,idaviso,idpostulante,mes_postulacion,sepostulo,idpais,titulo,descripcion,nombre_zona,...,nivel_laboral,nombre_area,denominacion_empresa,Unnamed: 0,mes_vista,dia_vista,hora_vistas,distancia_dia,distancia_hora,distancia_mes
14436245,21,14,1112451341,wVk188w,2,0,1,VENDEDOR DE MOSTRADOR,- VENTA DE PINTURAS Y ACCESORIOS- ACOMODAR PED...,Gran Buenos Aires,...,Junior,Ventas,Pintureria Fragueiro,NaN,NaN,NaN,NaN,100.0,100.0,100.0
13959817,9,23,1112312559,ZDPkJLY,3,0,1,Asesor inmobiliario profesional CABA,<p>Para reconocida y prestigiosa empresa de de...,Gran Buenos Aires,...,Senior / Semi-Senior,Ventas,Mook´s Recursos Humanos,NaN,NaN,NaN,NaN,100.0,100.0,100.0
533654,25,16,1112349943,KBAAdKQ,2,1,1,"Secretaria de Gerencia general, Empresa en Cab...",<p><strong>Importante empresa de la Zona de Ca...,Gran Buenos Aires,...,Senior / Semi-Senior,Secretaria,Solution,210559.0,2.0,25.0,21.0,0.0,-5.0,0.0
12330279,17,18,1112286083,xkd9K1a,2,1,1,J/ Operarios alimenticios - Zona Oeste,<p>Nos encontramos en la búsqueda de operarios...,Gran Buenos Aires,...,Senior / Semi-Senior,Producción,Grupo Gestión,NaN,NaN,NaN,NaN,100.0,100.0,100.0
11555493,16,6,1112411367,avqNVJ,1,0,1,OFICIAL ALUMINIO,Empresa dedicada a la fabricacion de aberturas...,Gran Buenos Aires,...,Senior / Semi-Senior,Oficios y Profesiones,turnkey aberturas,NaN,NaN,NaN,NaN,100.0,100.0,100.0
13540165,10,7,1112234178,vVDZwqE,3,0,1,Colorista para Laboratorio,"<p align=""center"">Importante Compañía Multinac...",Gran Buenos Aires,...,Senior / Semi-Senior,Ingeniería Química,ARRIVE RRHH,NaN,NaN,NaN,NaN,100.0,100.0,100.0
10206067,13,8,1112452111,16OvmL,4,1,1,Cajero/a,"<p><strong>FUNCION</strong></p><p><span lang=""...",Gran Buenos Aires,...,Otro,Comercial,BvhMas!,1823377.0,4.0,13.0,12.0,0.0,-4.0,0.0
5113071,7,23,1112278128,6rBveVO,2,1,1,GESTORES DE COBRANZAS - PART TIME,"<p align=""center"">¿SOS UN EXPERTO EN COBRANZAS...",Gran Buenos Aires,...,Senior / Semi-Senior,Oficios y Profesiones,MO&PC Collections Argentina S.R.L,NaN,NaN,NaN,NaN,100.0,100.0,100.0
11788773,14,7,1112307158,MVOkKDv,4,0,1,2 Choferes tipo Chasis y Balancín Zona Mercado...,<p>Para empresa logística de mercaderías refig...,Gran Buenos Aires,...,Senior / Semi-Senior,Transporte,Grupo CETA,NaN,NaN,NaN,NaN,100.0,100.0,100.0
11421670,1,17,1112235023,3NzpAqP,2,0,1,Analista Contable semi-senior de SOX y reporting,"<p style="""">Para importante industria de entre...",Gran Buenos Aires,...,Senior / Semi-Senior,Contabilidad,RANDSTAD,NaN,NaN,NaN,NaN,100.0,100.0,100.0


In [22]:
postulaciones = postulaciones[['idaviso','idpostulante','distancia_dia','distancia_hora','distancia_mes','sepostulo']]
postulaciones.head()

,idaviso,idpostulante,distancia_dia,distancia_hora,distancia_mes,sepostulo
0,1112257047,NM5M,100.0,100.0,100.0,1
1,1112257047,1kJqGb,100.0,100.0,100.0,1
2,1112257047,eOE9Rr,100.0,100.0,100.0,1
3,1112257047,Zrx8Xz,100.0,100.0,100.0,1
4,1112257047,ZrKNQY,100.0,100.0,100.0,1


In [23]:
train, test=train_test_split(postulaciones,test_size=0.10)
print("Train: ",len(train),"Test: ",len(test))

Train:  13900850 Test:  1544539


In [24]:
features=list(postulaciones.columns)
features

['idaviso',
 'idpostulante',
 'distancia_dia',
 'distancia_hora',
 'distancia_mes',
 'sepostulo']

In [25]:
features.remove('idaviso')
features.remove('idpostulante')
features

['distancia_dia', 'distancia_hora', 'distancia_mes', 'sepostulo']

In [26]:
features.remove('sepostulo')

In [27]:
x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']

In [28]:
from xgboost import XGBClassifier
c=XGBClassifier(objective='binary:logistic',max_depth=13,learning_rate=0.1,n_estimators=10,silent=False,n_jobs=-1)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl"
dt=c.fit(x_train,y_train)

In [29]:
score=c.score(x_test,y_test)*100
print(score)

70.19220621816606


/home/uriel/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [30]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.sample(10)

In [31]:
print(len(prediccion))

141819


In [32]:
print(len(postulantes))
print(len(avisos))
print(len(vistas))

504407
25901
16754298
